# Análise Preditiva de Churn com Dados do Google Sheets

Este notebook lê dados de chamados do Google Sheets, faz uma análise preditiva para identificar risco de churn e grava os resultados em uma nova aba da mesma planilha. Ideal para rodar no Google Colab.

## Autenticação com Google Sheets

Execute a célula abaixo para autenticar sua conta Google e permitir acesso ao Google Sheets.

In [ ]:
# Autenticação Google para Google Sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

## Leitura dos Dados do Google Sheets

Configure o ID da sua planilha e o nome da aba com os dados de chamados. Os dados serão lidos em um DataFrame pandas.

In [1]:
# Substitua pelos seus dados
SHEET_ID = 'COLE_AQUI_O_ID_DA_PLANILHA'
SHEET_NAME = 'chamados'  # Nome da aba com os dados

import pandas as pd
worksheet = gc.open_by_key(SHEET_ID).worksheet(SHEET_NAME)
data = worksheet.get_all_records()
df = pd.DataFrame(data)
df.head()

ModuleNotFoundError: No module named 'pandas'

## Análise Exploratória dos Dados

Visualize as principais estatísticas e distribuições dos chamados para entender o comportamento dos dados.

In [ ]:
# Estatísticas básicas
df.describe()

# Distribuição de churn (se houver coluna 'churn')
if 'churn' in df.columns:
    print(df['churn'].value_counts())
    df['churn'].value_counts().plot(kind='bar', title='Distribuição de Churn')

## Treinamento do Modelo Preditivo

Aqui vamos preparar os dados e treinar um modelo simples para prever churn com base nos chamados.

In [ ]:
# Exemplo: Treinando um modelo de classificação simples
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Supondo que 'churn' seja a coluna alvo e o resto são features
if 'churn' in df.columns:
    X = df.drop(['churn', 'id'], axis=1, errors='ignore')
    y = df['churn']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

## Gerar Lista de IDs de Chamados com Alto Risco de Churn

Aqui vamos identificar os chamados com maior probabilidade de churn e salvar os IDs para uso no Apps Script.

In [ ]:
# Exemplo: IDs com maior probabilidade de churn
if 'churn' in df.columns:
    # Supondo que o modelo foi treinado e temos X para todos os dados
    if 'id' in df.columns:
        probs = model.predict_proba(X)[:,1]
        df['churn_prob'] = probs
        high_risk = df[df['churn_prob'] > 0.7]  # Ajuste o threshold conforme necessário
        ids_high_risk = high_risk['id'].tolist()
        print(f'Chamados de alto risco: {ids_high_risk}')
        # Salvar em nova aba
        sh = gc.open_by_key(SHEET_ID)
        try:
            sh.add_worksheet(title='alto_risco', rows=len(ids_high_risk)+1, cols=2)
        except:
            pass  # Aba já existe
        ws_risk = sh.worksheet('alto_risco')
        ws_risk.clear()
        ws_risk.append_row(['id', 'churn_prob'])
        for _, row in high_risk.iterrows():
            ws_risk.append_row([row['id'], row['churn_prob']])

## Próximos Passos: Usando os Resultados no Apps Script

Agora, utilize a aba `alto_risco` da sua planilha Google para que o Apps Script envie alertas apenas para os chamados identificados como alto risco e recentes.